In [38]:
import pandas as pd
import os
import numpy as np

In [49]:
#specify num models
num_models = 5
model_dir = "../ensem_models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

res_dir = "../results_ensem"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)

models = os.listdir(f"{model_dir}")
models = sorted(models)

Store all predictions in one dataframe

In [50]:
# combine model predictions in a single df
for i in range(0, num_models):
    curr_csv = models[i]

    # makes sure that column names identify the model
    model_name = curr_csv.split('.')[0]
    temp = pd.read_csv(f"{model_dir}/{curr_csv}")
    temp.set_index('id', inplace=True)
    temp.rename(columns={"label": f"label_{model_name}"}, inplace=True)
    if i == 0:
        combined = temp
    else:
        combined = pd.concat([combined, temp], axis=1)
combined

,label_model1,label_model2,label_model3,label_model4,label_model5
id,,,,,
1,5,5,107,107,107
2,146,146,158,158,158
3,147,147,108,108,108
4,92,92,33,33,33
5,31,31,158,158,158
...,...,...,...,...,...
3996,172,172,106,106,106
3997,38,38,106,106,106
3998,103,103,10,10,10


Select mode

In [51]:
# function to select random mode in case of tie
def random_mode(row):
    modes = row.mode()
    return np.random.choice(modes)

# determine the mode across model predictions
combined['label'] = combined.apply(random_mode, axis=1).astype(int)
combined

,label_model1,label_model2,label_model3,label_model4,label_model5,label
id,,,,,,
1,5,5,107,107,107,107
2,146,146,158,158,158,158
3,147,147,108,108,108,108
4,92,92,33,33,33,33
5,31,31,158,158,158,158
...,...,...,...,...,...,...
3996,172,172,106,106,106,106
3997,38,38,106,106,106,106
3998,103,103,10,10,10,10


Store results in two ways

In [52]:
# store result, which contains all model predictions and the final label
combined.to_csv(f"{res_dir}/all_pred.csv")

# drop columns of individual model predictions
for i in range(0, num_models):
    curr_csv = models[i]

    # makes sure that column names identify the model
    model_name = curr_csv.split('.')[0]
    combined.drop(columns=[f"label_{model_name}"], inplace=True)

# store result, which is final prediction after ensembling
combined.to_csv(f"{res_dir}/ensemble_submission.csv")

## Diff method

In [ ]:
import torch
import torch.nn as nn

class EnsembleModel(nn.Module):   
    def __init__(self, modelList):
        super().__init__()
        self.models = nn.ModuleList(modelList)
        self.classifier = nn.Linear(200 * len(modelList), 200)
        
    def forward(self, x):
        outputs = []
        for model in self.models:
            outputs.append(model(x))
        
        x_cat = torch.cat(outputs, dim=1)
        out = self.classifier(x_cat)
        return out


# Load the models
models = [model1, model2, model3]
ensemble_model = EnsembleModel(models)

# freeze parameters of individual models
for param in ensemble_model.parameters():
    param.requires_grad = False

# unfreeze parameters of the classifier
for param in ensemble_model.classifier.parameters():
    param.requires_grad = True    


ensemble_model = ensemble_model.to(device)